# Fine-tuning SmolVLM Using Direct Preference Optimization (DPO) with TRL on a consumer GPU

In this example, we will finetune a smol **Vision Language Model (VLM)** with **Direct Preference Optimization (DPO)** using the Transformer Reinforcement Learning (TRL) library on consumer-grade GPUs.

We will finetune [`SmolVLM`](https://huggingface.co/blog/smolvlm) using a **preference dataset** to help the model align with desired outputs. The dataset we will use is [`HuggingFaceH4/rlaif-v_formatted`](https://huggingface.co/datasets/HuggingFaceH4/rlaif-v_formatted), which contains pairs of `prompt + image` along with a `chosen` and `rejected` answer for each pair. The goal of this finetuning process is to make the model consistently prefer the `chosen` answer from the dataset, reducing hullucinations.

## Setups

In [ ]:
!pip install  -U -q transformers trl datasets bitsandbytes peft accelerate
# Tested with transformers==4.46.3, trl==0.12.2, datasets==3.2.0, bitsandbytes==0.45.0, peft==0.14.0, accelerate==1.2.0

In [ ]:
!pip install -q flash-attn --no-build-isolation

## Load dataset

We will load the [`HuggingFaceH4/rlaif-v_formatted`](https://huggingface.co/datasets/HuggingFaceH4/rlaif-v_formatted) dataset.

In [ ]:
from datasets import load_dataset

dataset_id = "HuggingFaceH4/rlaif-v_formatted"
train_dataset, test_dataset = load_dataset(
    dataset_id,
    split=['train[:6%]', 'test[:1%]']
)

We also need to ensure all the images are RGB formatted:

In [ ]:
from PIL import Image

def ensure_rgb(example):
    image = example['images'][0]
    if isinstance(image, Image.Image):
        if image.mode != 'RGB':
            image = image.convert('RGB')
        example['images'] = [image]

    return example

In [ ]:
train_dataset = train_dataset.map(ensure_rgb, num_proc=4)
test_dataset = test_dataset.map(ensure_rgb, num_proc=4)

In [ ]:
train_dataset[0]

In [ ]:
train_dataset[0]['images'][0]

## Finetune the model using TRL

### Load the quantized model for training

In [ ]:
from transformers import AutoProcessor, Idefics3ForConditionalGeneration, BitsAndBytesConfig
import torch

model_id = 'HuggingFaceTB/SmolVLM-Instruct'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

processor = AutoProcessor.from_pretrained(model_id)
model = Idefics3ForConditionalGeneration.from_pretrained(
    model_id,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    _attn_implementation='flash_attention_2'
)

### Set up Q-LoRA and DPOConfig

In [ ]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.1,
    target_modules=['down_proj', 'up_proj', 'gate_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj'],
    use_dora=True,
    init_lora_weights='gaussian'
)

peft_model = get_peft_model(model, peft_config)

peft_model.print_trainable_parameters()

Now we will configure the training options using `DPOConfig`.

In [ ]:
from trl import DPOConfig

training_args = DPOConfig(
    output_dir='smolvlm-instruct-trl-dpo-rlaif-v',
    bf16=True,
    gradient_checkpointing=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=32,
    num_train_epochs=5,
    dataset_num_proc=8, # tokenization will use 8 processes
    dataloader_num_workers=8, # dataloading will use 8 workers
    logging_steps=10,
    report_to='tensorboard',
    push_to_hub=False,
    save_strategy='steps',
    save_steps=10,
    save_total_limit=1,
    eval_steps=10,
    eval_strategy='steps'
)

NExt, we will define the `DPOTrainer`.

**DPO** uses labeled preference data to guide the model toward generating responses that align with preferences. TRL's `DPOTrainer` will **tokenize the dataset** before training and save it to disk. This process can consume significant disk space, depending on the amount of data used for training.

In [ ]:
from trl import DPOTrainer

trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    tokenizer=processor
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model(training_args.output_dir)

## Test the finetuned model

In [ ]:
import gc
import time


def clear_memory():
    # Delete variables if they exist in the current global scope
    if "inputs" in globals():
        del globals()["inputs"]
    if "model" in globals():
        del globals()["model"]
    if "processor" in globals():
        del globals()["processor"]
    if "trainer" in globals():
        del globals()["trainer"]
    if "peft_model" in globals():
        del globals()["peft_model"]
    if "bnb_config" in globals():
        del globals()["bnb_config"]
    time.sleep(2)

    # Garbage collection and clearing CUDA memory
    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")


clear_memory()

We will reload the base model.

In [ ]:
processor = AutoProcessor.from_pretrained(model_id)
model = Idefics3ForConditionalGeneration.from_pretrained(
    model_id,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    _attn_implementation='flash_attention_2'
)

Next, we will attach the trained adapter to the pretrained model.

In [ ]:
adapter_path = "sergiopaniego/smolvlm-instruct-trl-dpo-rlaif-v"
model.load_adapter(adapter_path)

In [ ]:
# test
sample = test_dataset[0]
sample

In [ ]:
sample['images'][0]

We need to create a function to streamline the test process.

In [ ]:
def generate_text_from_sample(model, processor, sample, max_new_tokens=1024, device='cuda'):
    text_input = processor.apply_chat_template(
        sample['prompt'],
        add_generation_prompt=True
    )

    image_inputs = []
    image = sample['images'][0]
    if image.mode != 'RGB':
        image = image.convert('RGB')
    image_inputs.append([image])

    # Prepare the inputs for the model
    model_inputs = processor(
        text=text_input,
        images=image_inputs,
        return_tensors='pt'
    ).to(device)

    # Generate ids
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens
    )

    # Trim the generated ids
    generated_ids_trimmed = [
        out_ids[len(in_ids) :]
        for in_ids, out_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # Decode the output text
    output_text = processor.batch_decode(
        generated_ids_trimmed,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    return output_text[0]

In [ ]:
output = generate_text_from_sample(model, processor, sample)
output